<a href="https://colab.research.google.com/github/jajisan/Smart-Finder-An-AI-Powered-Skill-Gap-Analyzer-Career-Recommender/blob/main/Smart_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Sample user input
user_skills = ["Python", "Machine Learning", "Data Analysis", "SQL"]

# Sample job roles and required skills
job_data = {
    "Job Role": [
        "Data Analyst",
        "Machine Learning Engineer",
        "Backend Developer",
        "Business Analyst"
    ],
    "Required Skills": [
        "Python, SQL, Excel, Data Visualization",
        "Python, Machine Learning, Deep Learning, NLP",
        "Python, Django, REST API, SQL",
        "Excel, Power BI, Data Analysis, Communication"
    ]
}

job_df = pd.DataFrame(job_data)

# Sample course database
course_data = {
    "Skill": [
        "Excel", "Data Visualization", "Deep Learning", "NLP",
        "Django", "REST API", "Power BI", "Communication"
    ],
    "Recommended Course": [
        "Excel Basics on Coursera",
        "DataViz with Tableau - Udemy",
        "Deep Learning Specialization - Coursera",
        "Intro to NLP - edX",
        "Fullstack Django - Udemy",
        "API Development - FreeCodeCamp",
        "Power BI Bootcamp - Coursera",
        "Effective Communication - LinkedIn Learning"
    ]
}
course_df = pd.DataFrame(course_data)

# Cosine Similarity Matching ---

def preprocess_skills(skills_list):
    return ",".join(skills_list)

user_skills_str = preprocess_skills(user_skills)
all_skills = job_df["Required Skills"].tolist() + [user_skills_str]

vectorizer = CountVectorizer()
skill_matrix = vectorizer.fit_transform(all_skills)
cosine_sim = cosine_similarity(skill_matrix)

# Get similarity scores for user vs each job
similarities = cosine_sim[-1][:-1]  # last row vs all others
job_df["Match Score"] = similarities
job_df_sorted = job_df.sort_values(by="Match Score", ascending=False)

# Display top matches
print("Top Matching Job Roles:\n")
print(job_df_sorted[["Job Role", "Match Score"]])

# Skill Gap Analysis ---

def find_missing_skills(user_skills, job_skills):
    job_skills_set = set([s.strip().lower() for s in job_skills.split(",")])
    user_skills_set = set([s.strip().lower() for s in user_skills])
    missing = job_skills_set - user_skills_set
    return list(missing)

print("\nSkill Gaps:\n")
for index, row in job_df_sorted.iterrows():
    gaps = find_missing_skills(user_skills, row["Required Skills"])
    print(f"{row['Job Role']}: {gaps}")

# Course Recommendation ---

def recommend_courses(missing_skills, course_df):
    recommendations = []
    for skill in missing_skills:
        match = course_df[course_df["Skill"].str.lower() == skill]
        if not match.empty:
            recommendations.append(match["Recommended Course"].values[0])
    return recommendations

print("\nCourse Recommendations Based on Gaps:\n")
for index, row in job_df_sorted.iterrows():
    gaps = find_missing_skills(user_skills, row["Required Skills"])
    recs = recommend_courses(gaps, course_df)
    print(f"{row['Job Role']}:\n - " + "\n - ".join(recs) if recs else "No recommendations found")
    print()


Top Matching Job Roles:

                    Job Role  Match Score
1  Machine Learning Engineer     0.577350
0               Data Analyst     0.547723
2          Backend Developer     0.365148
3           Business Analyst     0.333333

Skill Gaps:

Machine Learning Engineer: ['nlp', 'deep learning']
Data Analyst: ['excel', 'data visualization']
Backend Developer: ['django', 'rest api']
Business Analyst: ['communication', 'power bi', 'excel']

Course Recommendations Based on Gaps:

Machine Learning Engineer:
 - Intro to NLP - edX
 - Deep Learning Specialization - Coursera

Data Analyst:
 - Excel Basics on Coursera
 - DataViz with Tableau - Udemy

Backend Developer:
 - Fullstack Django - Udemy
 - API Development - FreeCodeCamp

Business Analyst:
 - Effective Communication - LinkedIn Learning
 - Power BI Bootcamp - Coursera
 - Excel Basics on Coursera

